# О проекте


Задание относится к проекту UNICEF — международного подразделения ООН, чья миссия состоит в повышении уровня благополучия детей по всему миру. 


## Цель проекта

Отследить влияние условий жизни учащихся в возрасте от 15 до 22 лет на их успеваемость по математике, чтобы на ранней стадии выявлять студентов, находящихся в группе риска.

## Средства достижения целей

Реализовать достижение целей можно с помощью модели, которая предсказывала бы результаты госэкзамена по математике для каждого ученика школы.

## Задача проекта

Чтобы определиться с параметрами будущей модели, нужно провести разведывательный анализ данных и составьте отчёт по его результатам.

### Импорт библиотек

In [ ]:
import warnings; warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import os
import re
import math

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from itertools import combinations

from scipy.stats import ttest_ind
from scipy.stats import norm
from scipy.stats import t
from statsmodels.stats import weightstats 


%matplotlib inline

### Инструменты для предобработки

In [ ]:
# 
def column_content_analysis(column_name_list, column_type='str', more_than = 10):
    '''
        Функция для определения типовых параметров данных, а также определние "замусоренности".
        На вход принимает список столбцов датафрейма
    '''
    # проходим циклом по списку колонок    
    for column_name in column_name_list:
        print('-'*60)
        print('|\t' , 'Отчет по колонке [', column_name, ']', '\t'*5)
        print('-'*60)
        if column_name not in stud.columns:
            print('|  Внимание!!! Указанная колонка не найдена в датасете. \n')
            break

        col_count = stud[column_name].count()
        col_type = stud[column_name].dtype
        print('|  Тип данных:', col_type) 
        print('|  Заполнено значений:',  col_count, 'из', row_count)
        print('|  Значений для корректировки:', row_count - col_count)
        print('|  Полнота данных: ', round(col_count / row_count * 100 ,2), '%', sep='')
        print('|  Количество уникальных значений:', stud[column_name].nunique())

        print('|  Значений, встретившихся в столбце более', more_than,'раз:', 
              (stud[column_name].value_counts() > more_than).sum())

        if col_type == 'object':
            # Количество пробелов в начале в конце строки
            print('|  Количество раз пробелы найдены:')
            find_list = [len(re.findall(r'^\s+\w?', str(x))) for x in stud[column_name]]
            print('|    в начале строки:', sum(find_list))
            
            find_list = [len(re.findall(r'\w?\s+$', str(x))) for x in stud[column_name]]
            print('|    в конце  строки:', sum(find_list))
            
            find_list = [len(re.findall(r'[^\s\d\w]', str(x))) for x in stud[column_name]]
            print('|  Количество найденных специальных символов:', sum(find_list))
        
#         if col_type in ['int', 'float']:
            #
#             min_values = stud[column_name].min()
#             print('|  Минимальное значение:', round(min_values ,2))
            
#             max_values = stud[column_name].max()
#             print('|  Максимальное значение:', round(max_values, 2))
            
#             mean_values = stud[column_name].mean()
#             print('|  Среднее значение:', round(mean_values, 2))
            
#             median_values = stud[column_name].median()
#             print('|  Медианное значение:', round(median_values, 2))
         
        print('|  Уникальные значения:', stud[column_name].unique() , '\n')
    # end function




# column_content_analysis(['age'])





### Загрузка первичных данных

In [ ]:
# 
incoming_files = 'stud_math.csv'

# 
if incoming_files not in os.listdir():
    print('\n!!! ВНИМАНИЕ !!!'*5)
    print('Не найден входной файл данных:', incoming_files)
else:
    print(r'[OK] Успешно пройдена проверка наличия входного файла датасета.')


In [ ]:
# 
stud = pd.read_csv(incoming_files)

# 
row_count, col_count = stud.shape
row_count, col_count

In [ ]:
# показывать больше строк
pd.set_option('display.max_rows', 50)

# показывать больше колонок
pd.set_option('display.max_columns', col_count + 5) 

In [ ]:
stud.sample(9)

In [ ]:
stud.info()

### Первичная обработка данных

In [ ]:
# Удаляем столбец не заявленный в описании
if 'studytime, granular' in stud.columns:
    stud.drop(['studytime, granular'], inplace=True, axis = 1)

# приводим все названия колонок к нижнему регистру
stud.columns = [str.lower(x) for x in stud.columns]


all_columns = stud.columns

In [ ]:
# dtypes: float64(11), int64(1), object(17)

dig_columns = ['age','medu', 'fedu', 'traveltime', 'studytime', 'failures',
              'famrel', 'freetime', 'goout', 'health', 'absences', 'score']

stud.loc[:, dig_columns].info()

In [ ]:
# 
other_columns = [x for x in all_columns if x not in dig_columns]
other_columns

In [ ]:
# произведем замену пустых значений и значений nan в строковых столбцах
for col_name in other_columns:
    stud[col_name] = stud[col_name].apply(lambda x : None if pd.isnull(x)
                                    else None if x == 'nan'
                                    else None if x.strip == ''
                                    else x.strip())

In [ ]:
# произведем замену пустых значений и значений nan в числовых столбцах
for col_name in dig_columns:
    stud[col_name] = stud[col_name].apply(lambda x : None if pd.isnull(x)
                                    else None if x == 'nan'
                                    else x)

In [ ]:
# column_content_analysis(['address', 'age', 'medu'])

column_content_analysis(all_columns)

In [ ]:
stud.sample(9)

### Распределение признака для числовых переменных

In [ ]:
# количество числовых переменных
len(dig_columns)

In [ ]:
# список числовых столбцов
dig_columns

### age

In [ ]:
column_content_analysis(['age'])

In [ ]:
stud['age'].describe()

In [ ]:
stud['age'].hist()

In [ ]:
# age

In [ ]:
column_content_analysis(['age'])

In [ ]:
stud['age'].describe()

In [ ]:
stud['age'].hist()

In [ ]:
# medu

In [ ]:
column_content_analysis(['medu'])

In [ ]:
stud['medu'].describe()

In [ ]:
stud['medu'].hist()

In [ ]:
# 

In [ ]:
column_content_analysis(['fedu'])

In [ ]:
stud['fedu'].describe()

In [ ]:
stud['fedu'].hist()

In [ ]:
# 

In [ ]:
column_content_analysis(['traveltime'])

In [ ]:
stud['traveltime'].describe()

In [ ]:
stud['traveltime'].hist()

In [ ]:
column_content_analysis(['studytime'])

In [ ]:
stud['studytime'].describe()

In [ ]:
stud['studytime'].hist()

In [ ]:
column_content_analysis(['failures'])

In [ ]:
stud['failures'].describe()

In [ ]:
stud['failures'].hist()

In [ ]:
# 

In [ ]:
column_content_analysis(['famrel'])

In [ ]:
stud['famrel'].describe()

In [ ]:
stud['famrel'].hist()

In [ ]:
column_content_analysis(['freetime'])

In [ ]:
stud['freetime'].describe()

In [ ]:
stud['freetime'].hist()

In [ ]:
column_content_analysis(['goout'])

In [ ]:
stud['goout'].describe()

In [ ]:
stud['goout'].hist()

In [ ]:
column_content_analysis(['health'])

In [ ]:
stud['health'].describe()

In [ ]:
stud['health'].hist()

In [ ]:
column_content_analysis(['absences'])

In [ ]:
stud['absences'].describe()

In [ ]:
stud['absences'].hist()

In [ ]:
column_content_analysis(['score'])

In [ ]:
stud['score'].describe()

In [ ]:
stud['score'].hist()

### Устранение выбросов

### Оценка количества уникальных значений для номинативных переменных

### Преобразование данных

### Корреляционный анализ количественных элементов

### Отбор не коррелирующих переменных

### Анализ номинативных переменных, устранение не влияющих на предсказание конечной величины

In [ ]:
# на переменную scope

### Выводы о качестве данных

### Выводы о данных, которые подходят для построения дальнейшей модели

### Спасибо за внимание!